# HW 4
## 학과: 경영학부
## 학번: 1985084
## 이름: 임수지

### Problem 1

In [6]:
# Step 1. 파라미터 초기화
import torch

# 데이터
X1 = torch.tensor([0.83, -0.18, 0.27, 0.32, 0.25, -0.73, 0.19, -0.08, -0.47, -0.28,
                   -0.23, 0.25, 0.15, 0.92, 1.27, -0.50, 0.05, -0.74, -0.06, 0.70])
X2 = torch.tensor([0.63, -1.43, -1.35, 0.70, -1.48, 0.06, 1.14, 0.96, -1.65, 0.44,
                   0.62, 0.47, -0.08, 1.03, 1.24, -0.54, 0.08, 0.85, -1.53, 0.66])
Y  = torch.tensor([0.50, 1.34, 1.39, 0.40, 1.20, 0.11, -0.21, -0.26, 1.18, 0.59,
                   0.13, 0.17, 0.55, 0.32, 0.19, 0.46, 0.32, -0.35, 1.35, 0.20])

# 초기값 (보통 0 혹은 작은 랜덤값)
beta_0 = torch.randn(1, requires_grad=True)
beta_1 = torch.randn(1, requires_grad=True)
beta_2 = torch.randn(1, requires_grad=True)


In [7]:
# Step 2. 예측 함수 정의
def predict(X1, X2, beta_0, beta_1, beta_2):
    return beta_0 + beta_1 * X1 + beta_2 * X2

# Step 3. 손실함수 정의 (SSE)
def sse_loss(Y, Y_hat):
    return torch.sum((Y - Y_hat)**2)

# Step 4. 경사하강법 루프
lr = 0.01
n_epochs = 5000

for epoch in range(n_epochs):
    # forward
    Y_hat = predict(X1, X2, beta_0, beta_1, beta_2)
    loss = sse_loss(Y, Y_hat)

    # gradient 초기화
    loss.backward()

    # update (gradient descent)
    with torch.no_grad():
        beta_0 -= lr * beta_0.grad
        beta_1 -= lr * beta_1.grad
        beta_2 -= lr * beta_2.grad

    # gradient 초기화
    beta_0.grad.zero_()
    beta_1.grad.zero_()
    beta_2.grad.zero_()

    # 몇 번마다 출력
    if epoch % 500 == 0:
        print(f"Epoch {epoch:4d} | Loss = {loss.item():.4f}")


Epoch    0 | Loss = 7.4003
Epoch  500 | Loss = 0.4702
Epoch 1000 | Loss = 0.4702
Epoch 1500 | Loss = 0.4702
Epoch 2000 | Loss = 0.4702
Epoch 2500 | Loss = 0.4702
Epoch 3000 | Loss = 0.4702
Epoch 3500 | Loss = 0.4702
Epoch 4000 | Loss = 0.4702
Epoch 4500 | Loss = 0.4702


In [8]:
print("beta_0:", beta_0.item())
print("beta_1:", beta_1.item())
print("beta_2:", beta_2.item())

# 최종 예측값과 MSE 계산
Y_pred = predict(X1, X2, beta_0, beta_1, beta_2)
MSE = torch.mean((Y - Y_pred)**2)
print("MSE:", MSE.item())


beta_0: 0.4672396779060364
beta_1: 0.35831204056739807
beta_2: -0.5565071702003479
MSE: 0.023509198799729347


### Problem 2

In [9]:
# 2번
rate=1.3
torch.distributions.Exponential(rate, validate_args=None)

# 3번
import torch

rate = torch.tensor([1.3])
exp_dist = torch.distributions.Exponential(rate=rate)

x = torch.tensor([2.0])
cdf_x = exp_dist.cdf(x)
print("P(X <= 2.0) =", cdf_x.item())


P(X <= 2.0) = 0.9257264137268066


In [10]:
# 4번
import torch

# (a) 손으로 풀기 → 1번 손풀이 참고
# x0 = -torch.log(torch.tensor(0.05)) / 1.3 = 2.304

# (b) PyTorch icdf 사용
rate = torch.tensor([1.3])
exp_dist = torch.distributions.Exponential(rate=rate)
x0_icdf = exp_dist.icdf(torch.tensor([0.95]))
print("x0 (icdf) =", x0_icdf.item())  # 약 2.304 출력

# (c) 경사하강법(Gradient Descent)으로 최적화

# θ를 이용한 softplus 변환 (항상 양수)
theta = torch.tensor([1.0], requires_grad=True)  # softplus(1.0) ≈ 1.313
optimizer = torch.optim.SGD([theta], lr=0.05)

for step in range(100000):
    # softplus 변환: 항상 x0 > 0
    x0 = torch.log1p(torch.exp(theta))
    
    # 손실함수: (CDF(x0) - 0.95)^2
    loss = (exp_dist.cdf(x0) - 0.95)**2

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print("\n최종 추정값 (gradient descent):", x0.item())


x0 (icdf) = 2.3044092655181885

최종 추정값 (gradient descent): 2.304096221923828
